In [1]:
import matplotlib.pyplot as plt
import pandas as pd
from astropy.table import Table
import os
import numpy as np
# import cartopy.crs as ccrs
import json
import os
import seaborn as sns
from astropy.table import Table

In [2]:
os.chdir('/home/walml/repos/zoobot')

Pretend this is the final decals catalog

In [3]:
latest_export = pd.read_csv('/home/walml/Downloads/classifications_200520.csv')  # Tobias offline aggregated classifications

In [4]:
latest_export = latest_export.dropna(subset=['iauname'])  # a few test uploads don't have iaunames, we want to remove these (and would remove below anyway) but lets keep nan out of the iaunames ot remove set

In [5]:
to_upload = Table.read('/media/walml/beta/galaxy_zoo/decals/catalogs/dr5_nsa1_0_0_to_upload.fits').to_pandas()  # joint catalog, after selection cuts
to_upload['iauname'] = to_upload['iauname'].str.decode('utf8')



In [6]:
#dr1/2 raw classifications
dr1_c = pd.read_csv('/media/walml/beta/galaxy_zoo/decals/gzreduction_ouroborous/working_dir/raw/classifications/2017-10-15_galaxy_zoo_decals_classifications.csv')
dr2_c = pd.read_csv('/media/walml/beta/galaxy_zoo/decals/gzreduction_ouroborous/working_dir/raw/classifications/2017-10-15_galaxy_zoo_decals_dr2_classifications.csv')


## Remove galaxies with classifications that we shouldn't publish 

Because they don't have fits_filled

In [7]:
bad_iaunames = set()  

In [8]:
# and in decals, passed selection cuts, implicitly. Removes some test uploads, some of whcih have nan iaunames
good_images = latest_export['iauname'].isin(to_upload.query('fits_filled')['iauname'])  
good_images.value_counts()

True     227126
False     12556
Name: iauname, dtype: int64

In [9]:
to_upload.columns.values

array(['iauname', 'nsa_id', 'ra', 'dec', 'petrotheta', 'petroth50',
       'petroth90', 'z', 'fits_loc', 'png_loc', 'fits_ready',
       'fits_filled', 'png_ready'], dtype=object)

In [10]:
to_upload = to_upload.rename(index=str, columns={
    'fits_loc': 'local_fits_loc',
    'png_loc': 'local_png_loc'
})
                       

In [11]:
to_upload['local_png_loc']

0         b'/Volumes/EXTERNAL/decals/png/dr5/J094/J09465...
1         b'/Volumes/EXTERNAL/decals/png/dr5/J094/J09463...
2         b'/Volumes/EXTERNAL/decals/png/dr5/J094/J09463...
3         b'/Volumes/EXTERNAL/decals/png/dr5/J094/J09474...
4         b'/Volumes/EXTERNAL/decals/png/dr5/J094/J09475...
                                ...                        
343123    b'/Volumes/EXTERNAL/decals/png/dr5/J230/J23092...
343124    b'/Volumes/EXTERNAL/decals/png/dr5/J235/J23510...
343125    b'/Volumes/EXTERNAL/decals/png/dr5/J235/J23524...
343126    b'/Volumes/EXTERNAL/decals/png/dr5/J235/J23532...
343127    b'/Volumes/EXTERNAL/decals/png/dr5/J235/J23541...
Name: local_png_loc, Length: 343128, dtype: object

In [12]:
to_upload['png_loc'] = to_upload['local_png_loc'].str.decode('utf-8')
 

In [13]:
to_upload['png_loc'] = to_upload['png_loc'].str.replace('/Volumes/EXTERNAL/decals/png/', 'png_native/')  

In [14]:
to_upload['png_loc']

0         png_native/dr5/J094/J094651.40-010228.5.png
1         png_native/dr5/J094/J094630.85-004554.5.png
2         png_native/dr5/J094/J094631.59-005917.7.png
3         png_native/dr5/J094/J094744.18-004013.4.png
4         png_native/dr5/J094/J094751.74-003242.0.png
                             ...                     
343123    png_native/dr5/J230/J230924.60-001458.1.png
343124    png_native/dr5/J235/J235101.08-100042.7.png
343125    png_native/dr5/J235/J235247.10-105259.6.png
343126    png_native/dr5/J235/J235320.91-103238.7.png
343127    png_native/dr5/J235/J235414.90-101409.4.png
Name: png_loc, Length: 343128, dtype: object

In [15]:
to_upload.query('fits_filled').to_csv('final_dr5_uploadable_catalog.csv', index=False)  # use for master catalog

In [16]:
bad_iaunames = bad_iaunames.union(set(latest_export[~good_images]['iauname']))

In [17]:
len(bad_iaunames)

12556

In [18]:
np.sum(latest_export[good_images]['iauname'].duplicated())

0

In [19]:
# with open('data/bad_iaunames.json', 'w') as f:
#     json.dump(list(bad_iaunames), f)
# sent to Tobias and EXCLUDED from future aggregations

## Flag galaxies with earlier question versions (bulge, merger)

In [24]:
latest_export.columns.values

array(['iauname', 'smooth-or-featured_artifact',
       'smooth-or-featured_featured-or-disk', 'smooth-or-featured_smooth',
       'how-rounded_cigar-shaped', 'how-rounded_in-between',
       'how-rounded_round', 'disk-edge-on_no', 'disk-edge-on_yes',
       'edge-on-bulge_boxy', 'edge-on-bulge_none',
       'edge-on-bulge_rounded', 'bar_no', 'bar_weak', 'bar_strong',
       'has-spiral-arms_no', 'has-spiral-arms_yes',
       'spiral-winding_loose', 'spiral-winding_medium',
       'spiral-winding_tight', 'spiral-arm-count_1', 'spiral-arm-count_2',
       'spiral-arm-count_3', 'spiral-arm-count_4',
       'spiral-arm-count_cant-tell', 'spiral-arm-count_more-than-4',
       'bulge-size_none', 'bulge-size_small', 'bulge-size_moderate',
       'bulge-size_large', 'bulge-size_dominant', 'merging_none',
       'merging_minor-disturbance', 'merging_major-disturbance',
       'merging_merger', 'ra', 'dec', 'redshift', 'subject_id',
       'smooth-or-featured_total-votes', 'how-rounded_total-vo

In [28]:
(latest_export['smooth-or-featured_total-votes']- latest_export['merging_total-votes'] ).value_counts()

0.0      109089
1.0       54423
2.0       18165
3.0        8862
40.0       6718
          ...  
123.0         1
187.0         1
106.0         1
94.0          1
75.0          1
Length: 120, dtype: int64

In [20]:
merger_v1_answers = ['merging_both-v1', 'merging_neither-v1',
       'merging_tidal-debris-v1']
bulge_size_v1_answers = ['bulge-size_obvious-v1',
       'bulge-size_tiny-v1', 'bulge-size_none-v1']

In [21]:
has_v1_bulge_answers = latest_export[bulge_size_v1_answers].sum(axis=1) > 0
has_v1_bulge_answers.value_counts()

KeyError: "None of [Index(['bulge-size_obvious-v1', 'bulge-size_tiny-v1', 'bulge-size_none-v1'], dtype='object')] are in the [columns]"

Let's just drop that guy and not worry about it

In [ ]:
has_v1_merger_answers = latest_export[merger_v1_answers].sum(axis=1) > 0
has_v1_merger_answers.value_counts()

Enough of those that we should probably publish.

Check they only ever have v1, not both:

In [ ]:
merger_v2_answers = ['merging_none', # from renamed from neither
                     'merging_minor-disturbance', # new
                     'merging_major-disturbance', # new
                     'merging_merger'] # unchanged?

new_v2_answers = ['merging_none', # from renamed from neither
                     'merging_minor-disturbance', # new
                     'merging_major-disturbance']
shared_merger_answers = ['merging_merger']

# has_v2_merger_answers = latest_export[merger_v2_answers].sum(axis=1) > 0
# (has_v2_merger_answers & has_v1_merger_answers).sum()

has_new_merger_answers = latest_export[new_v2_answers].sum(axis=1) > 0

print(has_new_merger_answers.mean())
print(has_v1_merger_answers.mean())
print((has_new_merger_answers & has_v1_merger_answers).mean())

Okay, that's awkward - 5% of galaxies have answers for *both* v1 answers *and* the new v2 answers...
I think that for clarity it would be best to wipe the v1 answers for galaxies with both.

In [ ]:
has_both = has_new_merger_answers & has_v1_merger_answers

In [ ]:

# latest_export.loc[has_both, merger_v1_answers] = 0

# # check modifying slice still worked
# has_new_merger_answers = latest_export[new_v2_answers].sum(axis=1) > 0
# has_v1_merger_answers = latest_export[merger_v1_answers].sum(axis=1) > 0
# (has_new_merger_answers & has_v1_merger_answers).mean()

In [ ]:
assert False

# Flag galaxies by origin

In [ ]:
calibration_subject_sets = [
    19829,  # nair 250 each
    19832  # nair all
]

pre_active_subject_sets = [
    20036,  # first 3k
    20156, # 3k-5k,
    20603,  # 5k
    21156, # 10k
    51756, # 3k
    60259,  # 5k
    60995, # +5k   
    67383,  # +15k,
    67382,  # +500
    74807  # decals classic subjects, going to include as 'pre-active'
]

targeted_observations = [
    20354, # yjan 790
    60258  # yjan 1100ish (id/date is a touch strange)
]    

active_baseline = [74905]

active_priority = [74909]

# and will add the newly-found galaxies to active_baseline

In [ ]:
set_namer = {
    'calibration_nair_dr2_dr5': calibration_subject_sets,
    'pre_active': pre_active_subject_sets,
    'targeted': targeted_observations,
    'active_baseline': active_baseline,
    'active_priority': active_priority
}

I need to check the vote distribution for active learning vs random. If it worked right, there should be more featured galaxies in priority set.

In [ ]:
latest_subjects = pd.read_csv('/home/walml/repos/gz-panoptes-reduction/data/latest_subjects_export.csv')

In [ ]:
latest_subjects = latest_subjects[latest_subjects['workflow_id'].isin([6122.0, 10582.0, 10581.0])]  # decals public workflows
latest_subjects['workflow_id'].value_counts()

In [ ]:
latest_subjects = latest_subjects[~latest_subjects['subject_set_id'].isin([77652])]  # remove eagle subjects
latest_subjects['workflow_id'].value_counts()  # eagle is 15k in each of normal and enhanced workflows

In [ ]:
latest_subjects['subject_set_id'].value_counts()

In [ ]:
latest_subjects['metadata'] = latest_subjects['metadata'].apply(json.loads)

In [ ]:
def get_iauname(metadata):
    possible_keys = ['iauname', '!iauname', 'IAUNAME', '!IAUNAME']
    for key in possible_keys:
        if key in metadata.keys():
            return metadata[key]
    return np.nan

In [ ]:
latest_subjects['iauname'] = latest_subjects['metadata'].apply(get_iauname)

In [ ]:
pd.isna(latest_subjects['iauname']).sum()

In [ ]:
latest_subjects['subject_set_id'].value_counts()

In [ ]:
expected_sets = set(calibration_subject_sets + pre_active_subject_sets + targeted_observations + active_baseline + active_priority)
actual_sets = set(latest_subjects['subject_set_id'].unique())


In [ ]:
expected_sets

In [ ]:
actual_sets

In [ ]:
actual_sets.difference(expected_sets)

In [ ]:
# 74807 is an early active-learning-random upload, made just before turning active learning on, to the 'classic' (as opposed to enhanced) workflow
# they were pretty much all classified to N=100, probably because I hadn't yet got caesar working right on 'classic'
# about half of the galaxies (not subjects) are also in either pre-active-learning uploads or enhanced mode random uploads
# https://www.zooniverse.org/lab/5733/subject-sets/74807
# because they are random and highly classified, I list them in the data release as 'pre-active'

In [ ]:
classic_ids = latest_subjects.query('subject_set_id == "74807"')['subject_id']

In [ ]:
good_export = latest_export[good_images]
good_export[good_export['subject_id'].isin(classic_ids)]['smooth-or-featured_total-votes'].hist(bins=40)

In [ ]:
latest_subjects.query('subject_set_id == "74807"')['metadata'].iloc[5]

In [ ]:
latest_subjects.query('subject_set_id == "74905"')['metadata'].iloc[0]

In [ ]:
classic = latest_subjects.query('subject_set_id == "74807"')
in_both = classic[classic['iauname'].isin(latest_subjects.query('subject_set_id != "74807"')['iauname'])]
only_in_classic = classic[~classic['iauname'].isin(latest_subjects.query('subject_set_id != "74807"')['iauname'])]
len(classic), len(only_in_classic), len(in_both)

In [ ]:
latest_subjects[latest_subjects['iauname'].isin(in_both['iauname'])]['subject_set_id'].value_counts()  # which other subject sets are they in? random, (classic), a few active. Not calibration.

In [ ]:
latest_subjects.query('subject_set_id == "74807"')['locations'].iloc[0]

In [ ]:
expected_sets.difference(actual_sets)

In [ ]:
# latest_subjects = latest_subjects.query('subject_set_id != "74807"')

In [ ]:
latest_subjects['workflow_id'].value_counts()  # 74807 is simply the 'classic mode' galaxies, which nobody really classifies - note, 5765 length!

In [ ]:
def name_subject_set(subject_set_id):
    for key, ids in set_namer.items():
        if subject_set_id in ids:
            return key
    return 'failed'

# TODO when loading final catalog, use set namer above to add set name column

In [ ]:
latest_subjects['upload_group'] = latest_subjects['subject_set_id'].apply(lambda x: name_subject_set(int(x)))
latest_subjects['upload_group'].value_counts()

In [ ]:
latest_subjects.query('upload_group == "failed"')

In [ ]:
# assigning galaxies (iaunames) into subject sets may be awkward due to duplicates - check how many are in several

In [ ]:
t = latest_subjects.groupby('iauname').agg({'upload_group': lambda x: tuple(set(x))})

In [ ]:
t['upload_group'].value_counts()

Good - mostly the duplicates are all within the same subject sets. Let's resolve as follows:
- active_priority + active_baseline = active_priority
- calibration + anything = anything, calibration is not a random subset so preserve anything's selection func
- targeted + anything = anything, would have been in anything either way
- pre_active + active_priority = pre_active (should not have been reclassified)
- active_baseline + pre_active = pre_active (should not have been reclassified)

In [ ]:
# TODO actually name this

In [ ]:
latest_subjects.columns.values

In [ ]:
data = {json.loads(x)['0'] for x in latest_subjects['locations'][:3].values}
data

In [ ]:
def group_metadata(x):
    pass  # TODO

def group_locations(x):
    return json.dumps([json.loads(row)['0'] for row in x.values])  # list of unique urls, assuming 0 is the only key in each row

def group_upload_group(x):
    
    # only return calibration if that's the only set
#     if set(x) == set(['calibration_nair_dr2_dr5']):
#         return x.iloc[0]
# changed my mind - calibration will be flagged separately, and galaxies in and only in calibration are effectively early pre-active uploads
    if x == ['calibration_nair_dr2_dr5']:
        return 'pre_active'

    # drop calibration, will flag separately
    x_unique = set(x.values) - set(['calibration_nair_dr2_dr5'])
    if len(x_unique) == 1:  # just calibration or just one other subject set
        return list(x_unique)[0]

    if x_unique == set(('pre_active', 'active_baseline')):
        return 'pre_active'
    
    if x_unique == set(('active_baseline', 'active_priority')):
        return 'active_priority'
    
    if x_unique == set(('targeted', 'pre_active')):
        return 'pre_active'
    
    if x_unique == set(('active_priority', 'pre_active')):
        return 'pre_active'
    
    if x_unique == set(('active_priority', 'active_baseline')):
        return 'active_priority'
    
    if x_unique == set(('active_priority', 'active_baseline', 'pre_active')):
        return 'pre_active'

    raise ValueError('{}, {} not understood'.format(x, x_unique))


In [ ]:
# latest_subjects.groupby('iauname').agg({'locations': group_locations, 'upload_group': group_upload_group})

In [ ]:
grouped_subjects = latest_subjects.groupby('iauname').agg({'locations': group_locations, 'upload_group': group_upload_group}).reset_index()  # TODO add grouping of upload_groups

In [ ]:
grouped_subjects['upload_group'].value_counts()

In [ ]:
grouped_subjects['locations']

In [ ]:
# add calibration flag!
maybe_calibration = latest_subjects.groupby('iauname').agg({'upload_group': lamba x: 'calibration_nair_dr2_dr5' in x}).reset_index()  # TODO add grouping of upload_groups
maybe_calibration = maybe_calibration.rename(columns={'calibration_nair_dr2_dr5': 'dr2_dr5_nair_overlap'})

In [ ]:
final_df = pd.merge(latest_export[good_images], grouped_subjects, on='iauname', how='inner')


In [ ]:
len(final_df), len(latest_export[good_images])  # previously commented that will loose the five k 'bad metadata' galaxies from DR5 classic workflow, but I actually want to keep them?

In [ ]:
final_df = pd.merge(final_df, maybe_calibration, on='iauname', how='inner')

In [ ]:
len(final_df), len(latest_export[good_images])

In [ ]:
final_df.columns.values

In [ ]:
final_df['smooth-or-featured_total-votes'].sum()

In [ ]:
# merge in NSA later in merge_catalogs

In [ ]:
final_df.to_csv('current_final_dr5_result_without_metadata.csv', index=False)

In [ ]:
final_df.columns.values

In [ ]:
latest_subjects['counts'] = latest_subjects['subject_set_id']
iauname_counts_by_set = latest_subjects.groupby('subject_set_id').agg({'counts': 'count'})
iauname_counts_by_set.reset_index().sort_values('counts', ascending=False)

In [ ]:
iauname_counts_by_set.columns.values